# AICamp SageMaker image classification

## Test the API by sending a dog image


### John Burt
#### March 2020

### Dataset:
The [Stanford Dogs dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) contains images of 120 breeds of dogs from around the world. This dataset has been built using images and annotation from ImageNet for the task of fine-grained image categorization. Contents of this dataset (120 categories, 20580 images).

### Notebook purpose
Test the trained model instance, via an AWS Gateway API call. This notebook lets you select a local image file, prepares the image data, passes it to the model for inference, then displayes results. 


## Get class info from training lst file 

I'll use the directory name in the path to get dog breed name for each class.

In [33]:
import pandas as pd
import numpy as np

# lst file used for training
trainlstfile = 'dog_breeds_all_fold_1_train.lst'

df = pd.read_csv(trainlstfile, sep='\t', names=['sampid','classid','path'])
classnames = np.array([s.split('-')[1].split('/')[0] 
                       for s in df.groupby(by='classid').first().path])


## Filename selection dialog

In [34]:
%gui qt

from PyQt5.QtWidgets import QFileDialog

def gui_fname(dir=None, filters=None):
    """Select a file via a dialog and return the file name."""
    if dir is None: dir = './'
    if filters is None: filters = 'All files (*.*)'
    fname = QFileDialog.getOpenFileName(None, "Select file...", 
                dir, filter=filters)
    return fname[0]

## Select dog image, classify breed

- Use a file selection dialog to choose a dog image file in a local folder.

- Format and embed image into json payload object.

- Post to the image payload to the API gateway.

- Receive model results and select breed ID based on highest output value.

Note: I've modified the Lambda function to take multiple images in the form of a list. The API then returns a list of results. If I send a single image in the payload, the Lambda detects this and just does one classification, but still returns a list of results with one element.

In [46]:
import base64 # encode/decode image in base64
import json
import requests

# Collection of dog images not in training set
rootdir = 'C:/Users/john/notebooks/aicamp/dogs/test_images'

# select an image file to test
imgpath = gui_fname(dir=rootdir, filters='image (*.jpg)')

# read the image, convert to base64, embed in json payload object
with open(imgpath, 'rb') as image_file:
   encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
payload = json.dumps( {'body': encoded_string } )

# my breed prediction microservice API URL
api_url = 'https://uqquh6whab.execute-api.us-west-2.amazonaws.com/beta/classify'

# post the image, receive inference response
r = requests.post(url=api_url, data=payload, timeout=5)

try:
    # Classifier output is a list of results,
    # Since I sent one image, it will be the first list element
    classout = np.array(r.json()['body'])[0]

    # select highest output 
    selclass = np.argmax(classout)
    # sort by output, desc
    sortidx = np.argsort(-classout)
    print('Predicted dog breed, sorted by model output:')
    print('\nID\tOutput\tBreed name\n')
    for i, x, classname in zip(sortidx,classout[sortidx],classnames[sortidx]):
        print('%d   \t%1.3f %s\t%s'%(i,x,'*' if selclass==i else ' ', classname))
except:
    print('There was an error accessing the API.')
    print('Check:')
    print('  - Model endpoint is In Service')
    print('  - Lambda is using correct endpoint and is updated')
    print('  - API is active and updated')
    

Predicted dog breed, sorted by model output:

ID	Output	Breed name

2   	0.855 *	Maltese_dog
52   	0.034  	West_Highland_white_terrier
53   	0.019  	Lhasa
39   	0.014  	Sealyham_terrier
105   	0.011  	Great_Pyrenees
4   	0.006  	Shih
113   	0.005  	toy_poodle
71   	0.004  	kuvasz
78   	0.004  	Old_English_sheepdog
49   	0.004  	Tibetan_terrier
43   	0.003  	Dandie_Dinmont
65   	0.003  	clumber
3   	0.003  	Pekinese
106   	0.002  	Samoyed
114   	0.002  	miniature_poodle
45   	0.002  	miniature_schnauzer
1   	0.002  	Japanese_spaniel
51   	0.001  	soft
36   	0.001  	Yorkshire_terrier
6   	0.001  	papillon
77   	0.001  	komondor
5   	0.001  	Blenheim_spaniel
18   	0.001  	borzoi
37   	0.001  	wire
61   	0.001  	English_setter
102   	0.001  	pug
34   	0.001  	Norfolk_terrier
96   	0.001  	Saint_Bernard
57   	0.001  	Labrador_retriever
28   	0.001  	Staffordshire_bullterrier
107   	0.001  	Pomeranian
48   	0.001  	Scotch_terrier
64   	0.001  	Brittany_spaniel
115   	0.001  	standard_poodle
